In [ ]:
# Установка необходимых библиотек
!pip install PyPDF2[crypto]
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Выделение списка растений из атласа
q=[]
with open('work_directory/Атлас_2021.txt') as f:
    text = f.read()
    #matches = re.findall(r"[А-Я]+\n", text, re.MULTILINE)
    matches = re.findall(r"[А-Я]+(?:\s[А-Я]+)?\n", text)
    # matches = re.findall(r"([А-Я]+\s)+([А-Я]+)\n", text)
    #matches = re.findall(r"[А-Я]+\s[А-Я]+\n", text)
    matches = set(matches)
    #print(matches)
    for i in matches:
        if ' ' in i:
            q.append(i)
q = sorted(q)
regex = re.compile(r"^[А-Я]{1,2}\s")
filtered = [i[:-1] for i in q if not regex.search(i)]
print(filtered)


['АГАЛ ШЕРСТИСТОЦВЕТКОВЫЙ', 'АДЕНОСТИЛЕС РОМБОЛИСТНЫЙ', 'АИР ОБЫКНОВЕННЫЙ', 'АЛИЯ ВЫСОКАЯ', 'АЛОЭ ДРЕВОВИДНОЕ', 'АЛТЕЙ ЛЕКАРСТВЕННЫЙ', 'АММИ БОЛЬШАЯ', 'АММИ ЗУБНАЯ', 'АНЕЦ ОБЫКНОВЕННЫЙ', 'АНИС ОБЫКНОВЕННЫЙ', 'АНТУС РОЗОВЫЙ', 'АПИВА ДВУДОМНАЯ', 'АПОНТИКУМ САФЛОРОВИДНЫЙ', 'АРИЯ ИСЛАНДСКАЯ', 'АРНИКА ГОРНАЯ', 'АРНИКА ШАМИССО', 'АРОНИЯ ЧЕРНОПЛОДНАЯ', 'АСАВКА ОБЫКНОВЕННАЯ', 'АСТОРОПША ПЯТНИСТАЯ', 'БАГУЛЬНИК БОЛОТНЫЙ', 'БАДАН ТОЛСТОЛИСТНЫЙ', 'БАРБАРИС ОБЫКНОВЕННЫЙ', 'БАРВИНОК МАЛЫЙ', 'БАРХАТ АМУРСКИЙ', 'БЕДРЕНЕЦ КАМНЕЛОМКА', 'БЕЗВРЕМЕННИК ВЕЛИКОЛЕПНЫЙ', 'БЕЛЕНА ЧЕРНАЯ', 'БЕЛОКОПЫТНИК ГИБРИДНЫЙ', 'БЕССМЕРТНИК ПЕСЧАНЫЙ', 'БКА ЛЕКАРСТВЕННАЯ', 'БОРЕЦ СЕВЕРНЫЙ', 'БРИОФИЛЛУМ ПЕРИСТЫЙ', 'БРУСНИКА ОБЫКНОВЕННАЯ', 'БУЗИНА ЧЕРНАЯ', 'ВАЛЕРИАНА ЛЕКАРСТВЕННАЯ', 'ВАСИЛЕК БЛАГОСЛОВЕННЫЙ', 'ВАСИЛЕК ПОСЕВНОЙ', 'ВЕРБЕНА ЛЕКАРСТВЕННАЯ', 'ВЗДУТОПЛОДНИК СИБИРСКИЙ', 'ВИНОГР АДОЛИСТНЫЙ', 'ВОЛОДУШКА ЗОЛОТИСТАЯ', 'ГАМАМЕЛИС ВИРГИНСКИЙ', 'ГОРЕЦ ЗМЕИНЫЙ', 'ГОРЕЦ ПЕРЕЧНЫЙ', 'ГОРЕЦ ПОЧЕЧУЙНЫЙ', 'ГОРЕЦ ПТИЧИЙ', 'ГОРИЦВЕТ В

In [ ]:
text1 = ' '.join(text.split('\n'))

In [ ]:
# Разделение
def split_to_columns(text1, column_list):
  dict_ = {}
  for i in range(len(column_list)-1):
    r = re.findall(rf'{column_list[i]}(.*?){column_list[i+1]}', text1)
    if r:
        for j in r:
          if 'Описание' in j:
            dict_[column_list[i]] = j
            # print('*'*100)  
            # print(j)
  return dict_


In [ ]:
split_text1 = split_to_columns(text1, filtered)


In [ ]:
with open('plants.txt', 'w+') as f:   
    # write elements of list
    for items in split_text1.keys():
        f.write('%s\n' %items)

In [ ]:
abstracts = ['Описание.', 'Ареал.', 'Экология.', 'Ресурсы.', 'Химический состав.', 'Сырье.', 'Фармокологические свойства.', 'Применение в медицине.']

In [ ]:
keys = split_text1.keys()
plant_info = []
for i, key in enumerate(keys):
  details = {}
  for j in range(len(abstracts) - 1):
    row = split_text1[key]
    start_pos = split_text1[key].find(abstracts[j])
    end_pos = split_text1[key].find(abstracts[j+1])
    details['Название.'] = key
    details[abstracts[j]] = row[start_pos+len(abstracts[j]):end_pos].strip()
    if j == (len(abstracts) - 2):
      start_pos = row.find(abstracts[j+1])
      details['Применение в медицине.'] = row[start_pos+len(abstracts[j+1]):].strip()
      
  plant_info.append(details)

In [ ]:
plant_info[0].keys()

dict_keys(['Название.', 'Описание.', 'Ареал.', 'Экология.', 'Ресурсы.', 'Химический состав.', 'Сырье.', 'Фармокологические свойства.', 'Применение в медицине.'])

In [ ]:
df = pd.DataFrame(plant_info)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline

In [ ]:
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(dev)
device

device(type='cuda', index=0)

In [ ]:
qa_pipeline = pipeline(
     "question-answering",
     model="Den4ikAI/rubert_large_squad_2",
     tokenizer="Den4ikAI/rubert_large_squad_2",
     device=device)

In [ ]:
def predict(context, msg):
  prediction =  qa_pipeline({
      'context': context.lower(),
      'question': msg
  })
  return prediction['answer']

In [ ]:
df['Ресурсы.'] = df['Ресурсы.'].apply(lambda x: str(x).replace(' - ', ''))

In [ ]:
df['Фармокологические свойства.'] = df['Фармокологические свойства.'].apply(lambda x: str(x).replace(' - ', ''))

In [ ]:
df['Химический состав.'] = df['Химический состав.'].apply(lambda x: str(x).replace(' - ', '')) 

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Период сбора урожая'] = df['Ресурсы.'].apply(lambda x: predict(str(x), 'когда производят уборку урожая') if x else 'No context available')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
df['Потребность'] = df['Ресурсы.'].apply(lambda x: predict(str(x), 'ежегодная потребность сырья') if x else 'No context available')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
df['Период посева'] = df['Ресурсы.'].apply(lambda x: predict(str(x), 'когда производят посев') if x else 'No context available')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
df['Содержание в мед препаратах'] = df['Химический состав.'].apply(lambda x: predict(str(x), 'в каких медицинских препаратах содержится, наименование') if x else 'No context available')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
df['Основной компонент'] = df['Химический состав.'].apply(lambda x: predict(str(x), 'основной компонент') if x else 'No context available')

In [ ]:
df['Типы почв'] = df['Экология.'].apply(lambda x: predict(str(x), 'типы почв') if x else 'No context available')

In [ ]:
df.to_csv('table.csv')

In [ ]:
df

,Название.,Описание.,Ареал.,Экология.,Ресурсы.,Химический состав.,Сырье.,Фармокологические свойства.,Применение в медицине.,Период сбора урожая,Потребность,Период посева,Содержание в мед препаратах,Основной компонент
0,АГАЛ ШЕРСТИСТОЦВЕТКОВЫЙ,Летнезеленый травянистый многолетник. Корнева...,Астрагал шерстистоцветковый – причерномор - ск...,"Степной вид, обитающий на целинных участках. ...",Растение считается реликтовым. Его местопроизр...,"В траве содержатся флавоноиды, сумма которых ...",Качество сырья нормировалось Госу - дарственно...,l. Семейство бобовые (мотыльковые) – Fabaceae ...,Траву багульника в виде настоя используют при...,в сухую погоду в фазу цветения,около 20 % от массы свежесобранного,в начале цветения,луб и листья бархата амурского,бетулин
1,АДЕНОСТИЛЕС РОМБОЛИСТНЫЙ,Травянистый многолетник высотой 50-150 (250) ...,Аденостилес ромболистный – эндемик Кавказа. С...,Аденостилес растет в горно-лес - ном (преимуще...,Промысловые массивы аденостилеса приурочены к...,"ифиллиновый, крестовник ромболистный) Adenosty...","стовник платифиллиновый, крестовник ромболистн...","ый, крестовник ромболистный) Adenostyles rhom...","линовый, крестовник ромболистный) Adenostyles ...",кратковременно,имеют тенденцию к сокращению,при язвенной болезни желудка и двенадцатиперст...,"как в траве, так и в корневищах",алкалоид
2,АИР ОБЫКНОВЕННЫЙ,Многолетнее травянистое однодольное растение ...,"Ареал вида дизъюнктивный, состоит из трех фра...",Аир обыкновенный — прибрежно-во - дное растени...,До 90-х годов XX в. корневища аира заготовляли...,В корневищах содержится эфирное масло (до 6 %...,Качество сырья нормируется Государ - ственной ...,тво ароидные — Araceae Описание. Многолетнее ...,Препараты из корневищ аира применяются для ле...,"летом, с июня по сентябрь",100-120 т в год,"летом, с июня по сентябрь","борнеол, камфен",крахмал
3,АЛИЯ ВЫСОКАЯ,Травянистый многолетник высотой 50-150 (250) ...,Аденостилес ромболистный – эндемик Кавказа. С...,Аденостилес растет в горно-лес - ном (преимуще...,Промысловые массивы аденостилеса приурочены к...,В корневищах содержится эфирное масло (до 6 %...,Качество сырья нормируется Государ - ственной ...,.................................................,Препараты из корневищ аира применяются для ле...,ежегодно,не более 30 % общего их запаса,подвяливают на открытом воздухе,"борнеол, камфен",крахмал
4,АЛОЭ ДРЕВОВИДНОЕ,Суккулентный кустарник высотой до 5 м. В Росс...,Родина алоэ — Южная Африка и некото -рые остро...,ник) Alоe arborescens Mill. Семейство ксантор...,В России нет дикорастущего алоэ. В нашей стран...,arborescens Mill. Семейство ксанторреевые— Xa...,летник) Alоe arborescens Mill. Семейство ксан...,ens Mill. Семейство ксанторреевые— Xanthorrho...,Из свежих листьев алоэ выпускают препараты: а...,хорошо развитые нижние и средние листья,повышенной потребностью в железе,зимой,антрахиноны,сырьё
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,ЩАВЕЛЬ КОНСКИЙ,Многолетнее травянистое растение вы - сотой 60...,ель густой) Rumex confertus Willd . Семейство ...,R. confertus — мезофильное растение. Распрост...,Промысловые запасы выявлены на Алтае (в пойме ...,Rumex confertus Willd . Семейство гречишные — ...,ель густой) Rumex confertus Willd . Семейство ...,fertus Willd . Семейство гречишные — Polygonac...,Препараты конского ща - веля применяют для леч...,сенокосных угодий от этого растения,около 3 тыс. т.,осенью,стероиды; смолы,щавель
148,ЩИТОВНИК МУЖСКОЙ,тник мужской) Dryopteris filix-mas (L.) Schott...,оротник мужской) Dryopteris filix-mas (L.) Sch...,"Шитовник мужской — мезофильное, лесное растен...",Ежегодная потребность в сырье щитовника мужско...,кой) Dryopteris filix-mas (L.) Schott Семейств...,оротник мужской) Dryopteris filix-mas (L.) Sch...,pteris filix-mas (L.) Schott Семейство щитовни...,Способность препара - тов щитовника мужского (...,сушат в тени,450 т,осенью,корневища щитовника мужского,компонентный состав и биологическая активность
149,ЭВКАЛИПТ ПРУТОВИДНЫЙ,tus viminalis Labill . С

In [ ]:
df = pd.read_csv('table.csv', index_col=0)

In [ ]:
df

,Название.,Описание.,Ареал.,Экология.,Ресурсы.,Химический состав.,Сырье.,Фармокологические свойства.,Применение в медицине.,Период сбора урожая,Потребность,Период посева,Содержание в мед препаратах,Основной компонент
0,АГАЛ ШЕРСТИСТОЦВЕТКОВЫЙ,Летнезеленый травянистый многолетник. Корнева...,Астрагал шерстистоцветковый – причерномор - ск...,"Степной вид, обитающий на целинных участках. ...",Растение считается реликтовым. Его местопроизр...,"В траве содержатся флавоноиды, сумма которых ...",Качество сырья нормировалось Госу - дарственно...,l. Семейство бобовые (мотыльковые) – Fabaceae ...,Траву багульника в виде настоя используют при...,в сухую погоду в фазу цветения,около 20 % от массы свежесобранного,в начале цветения,луб и листья бархата амурского,бетулин
1,АДЕНОСТИЛЕС РОМБОЛИСТНЫЙ,Травянистый многолетник высотой 50-150 (250) ...,Аденостилес ромболистный – эндемик Кавказа. С...,Аденостилес растет в горно-лес - ном (преимуще...,Промысловые массивы аденостилеса приурочены к...,"ифиллиновый, крестовник ромболистный) Adenosty...","стовник платифиллиновый, крестовник ромболистн...","ый, крестовник ромболистный) Adenostyles rhom...","линовый, крестовник ромболистный) Adenostyles ...",кратковременно,имеют тенденцию к сокращению,при язвенной болезни желудка и двенадцатиперст...,"как в траве, так и в корневищах",алкалоид
2,АИР ОБЫКНОВЕННЫЙ,Многолетнее травянистое однодольное растение ...,"Ареал вида дизъюнктивный, состоит из трех фра...",Аир обыкновенный — прибрежно-во - дное растени...,До 90-х годов XX в. корневища аира заготовляли...,В корневищах содержится эфирное масло (до 6 %...,Качество сырья нормируется Государ - ственной ...,тво ароидные — Araceae Описание. Многолетнее ...,Препараты из корневищ аира применяются для ле...,"летом, с июня по сентябрь",100-120 т в год,"летом, с июня по сентябрь","борнеол, камфен",крахмал
3,АЛИЯ ВЫСОКАЯ,Травянистый многолетник высотой 50-150 (250) ...,Аденостилес ромболистный – эндемик Кавказа. С...,Аденостилес растет в горно-лес - ном (преимуще...,Промысловые массивы аденостилеса приурочены к...,В корневищах содержится эфирное масло (до 6 %...,Качество сырья нормируется Государ - ственной ...,.................................................,Препараты из корневищ аира применяются для ле...,ежегодно,не более 30 % общего их запаса,подвяливают на открытом воздухе,"борнеол, камфен",крахмал
4,АЛОЭ ДРЕВОВИДНОЕ,Суккулентный кустарник высотой до 5 м. В Росс...,Родина алоэ — Южная Африка и некото -рые остро...,ник) Alоe arborescens Mill. Семейство ксантор...,В России нет дикорастущего алоэ. В нашей стран...,arborescens Mill. Семейство ксанторреевые— Xa...,летник) Alоe arborescens Mill. Семейство ксан...,ens Mill. Семейство ксанторреевые— Xanthorrho...,Из свежих листьев алоэ выпускают препараты: а...,хорошо развитые нижние и средние листья,повышенной потребностью в железе,зимой,антрахиноны,сырьё
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,ЩАВЕЛЬ КОНСКИЙ,Многолетнее травянистое растение вы - сотой 60...,ель густой) Rumex confertus Willd . Семейство ...,R. confertus — мезофильное растение. Распрост...,Промысловые запасы выявлены на Алтае (в пойме ...,Rumex confertus Willd . Семейство гречишные — ...,ель густой) Rumex confertus Willd . Семейство ...,fertus Willd . Семейство гречишные — Polygonac...,Препараты конского ща - веля применяют для леч...,сенокосных угодий от этого растения,около 3 тыс. т.,осенью,стероиды; смолы,щавель
148,ЩИТОВНИК МУЖСКОЙ,тник мужской) Dryopteris filix-mas (L.) Schott...,оротник мужской) Dryopteris filix-mas (L.) Sch...,"Шитовник мужской — мезофильное, лесное растен...",Ежегодная потребность в сырье щитовника мужско...,кой) Dryopteris filix-mas (L.) Schott Семейств...,оротник мужской) Dryopteris filix-mas (L.) Sch...,pteris filix-mas (L.) Schott Семейство щитовни...,Способность препара - тов щитовника мужского (...,сушат в тени,450 т,осенью,корневища щитовника мужского,компонентный состав и биологическая активность
149,ЭВКАЛИПТ ПРУТОВИДНЫЙ,tus viminalis Labill . С

In [ ]:
def chemistry_compose(text):
  matches = re.findall(r'\b\w*(?:ин|ид)', text)
  return ' '.join([match_ for match_ in matches if len(match_) > 4])

In [ ]:
df['Содержание БАВ'] = df['Химический состав.'].apply(lambda x: chemistry_compose(str(x)))

In [ ]:
dff = pd.read_csv('region_herb.csv', sep='\t')
dff = dff.rename(columns={'Растение': 'Название.'})
dff['Название.'] = dff['Название.'].str.upper()
merged_df = pd.merge(df, dff, on='Название.')

In [ ]:
merged_df.to_csv('merged_df.csv')

In [ ]:
df.to_excel('result/result_table.xlsx', index= False, encoding= 'windows-1251')